<a href="https://colab.research.google.com/github/IshantheG/Filled/blob/main/Copy_of_Algal_Blooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import math
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from numpy import arange


Data Collection and Processing


In [ ]:
#loading the csv data to our pandas DataFrame
algae_data = pd.read_csv("/content/2021_WLE_Weekly_Datashare_CSV.csv-_1_.csv")

In [ ]:
algae_imp_data = algae_data.drop(columns = ['Date' ,'Wind_speed_knots', 'Wave_Ht_ft' , 'Station_Depth_m' , 'Sample_Depth_m', 'SpCond_uScm-1' , 'Arrival_Time' , 'Departure_Time' , 'Lat_deg' , 'Long_deg' , 'BeamAtten_m-1' ,	'Tra0miss_pct' ,	'DO_mgL-1' , 'PAR_uEcm-2s-1', 'Secchi_Depth_m', 'Site' , 'Sample_Depth_category', ], axis = 0)


In [ ]:
W = algae_imp_data['Extracted_CHLa_ugL-1'].round(decimals = 0)

In [ ]:
#print first 5 rows of the dataset
algae_imp_data.head()


,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
0,2,19.22,10.30,0.0,0.00,3.056700,8.91774
1,2,16.65,1.86,0.0,0.00,0.133553,36.26974
2,2,21.30,37.10,0.0,0.14,18.254920,76.77930
3,2,19.96,5.67,0.0,0.00,0.776635,10.75724
4,2,20.55,39.40,0.0,0.00,10.247903,54.15930


In [ ]:
#print last 5 rows of the datatset
algae_imp_data.tail()

,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
146,5,17.65,5.25,0.00,0.00,0.492174,3.411850
147,5,17.35,9.01,0.00,0.18,15.417222,13.226850
148,2,13.74,12.30,0.13,0.13,1.845166,9.489415
149,2,11.78,34.00,0.00,0.11,0.733771,2.245165
150,2,13.56,5.40,0.00,0.16,1.476077,7.344415


In [ ]:
#number of rows and colums in the dataset
algae_imp_data.shape

(151, 7)

In [ ]:
#getting some info about the data
algae_imp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sky                            151 non-null    int64  
 1   Temp_C                         151 non-null    float64
 2   Turbidity_NTU                  151 non-null    float64
 3   Particulate_Microcystin_ugL-1  151 non-null    float64
 4   Dissolved_Microcystin_ugL-1    151 non-null    float64
 5   Extracted_PC_ugL-1             151 non-null    float64
 6   Extracted_CHLa_ugL-1           151 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 8.4 KB


In [ ]:

#checking for missing values
algae_imp_data.isnull().sum()


Sky                              0
Temp_C                           0
Turbidity_NTU                    0
Particulate_Microcystin_ugL-1    0
Dissolved_Microcystin_ugL-1      0
Extracted_PC_ugL-1               0
Extracted_CHLa_ugL-1             0
dtype: int64

In [ ]:
#statiscal measures about the data
algae_imp_data.describe()


,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,3.536424,22.352848,9.906954,0.650397,0.135430,7.863269,22.313052
std,1.900087,3.533532,11.444194,1.241378,0.134768,12.478371,21.278869
min,0.000000,0.000000,1.130000,0.000000,0.000000,0.000000,1.052090
25%,2.000000,20.405000,3.875000,0.000000,0.000000,0.816407,8.186457
50%,5.000000,23.380000,6.820000,0.120000,0.140000,3.226893,14.494350
75%,5.000000,24.430000,11.050000,0.695000,0.220000,8.726029,28.042560
max,7.000000,27.670000,88.700000,7.280000,0.690000,74.906175,117.487500


In [ ]:
#checking the distribuiton of tragert varaibel
algae_imp_data['Extracted_CHLa_ugL-1'].value_counts()

8.826870     2
8.917740     1
14.299610    1
57.570825    1
71.025825    1
            ..
8.839870     1
9.028370     1
77.121525    1
23.204870    1
7.344415     1
Name: Extracted_CHLa_ugL-1, Length: 150, dtype: int64

Splitting the Features and Target


In [ ]:
X = algae_imp_data.drop(columns = ['Extracted_CHLa_ugL-1',], axis = 1)
Y = algae_imp_data['Extracted_CHLa_ugL-1']

In [ ]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
print(X)


[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]


In [ ]:
print(Y)

0       8.917740
1      36.269740
2      76.779300
3      10.757240
4      54.159300
         ...    
146     3.411850
147    13.226850
148     9.489415
149     2.245165
150     7.344415
Name: Extracted_CHLa_ugL-1, Length: 151, dtype: float64


Splitting the Data into training and Testing


In [ ]:
X_train = X.sample(frac = 0.8, random_state = 2)
X_test = X.drop(X_train.index)
Y_train = Y.sample(frac = 0.8, random_state = 2)
Y_test = Y.drop(Y_train.index)
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)



In [ ]:
print(Y_test)

4      54
7       5
15     12
22      7
31      6
33      4
37      4
38      5
43     33
46     14
47      8
51      9
58      5
63     21
67     34
68     28
69     13
70      8
72     65
75     84
88     26
95     18
102    27
103    23
104    48
107    47
116    18
124    11
138    10
146     3
Name: Extracted_CHLa_ugL-1, dtype: int64


In [ ]:
print(X_train)

     Sky  Temp_C  ...  Dissolved_Microcystin_ugL-1  Extracted_PC_ugL-1
85     5   25.71  ...                         0.16            5.275176
111    2   22.61  ...                         0.14           28.231623
3      2   19.96  ...                         0.00            0.776635
128    2   19.67  ...                         0.21            4.430225
6      2   16.67  ...                         0.00            0.173510
..   ...     ...  ...                          ...                 ...
80     5   25.92  ...                         0.30            7.274354
66     6   25.56  ...                         0.39           12.605717
139    2   19.86  ...                         0.19            2.189227
83     5   27.67  ...                         0.25            8.571110
143    5   17.54  ...                         0.10            2.447194

[121 rows x 6 columns]


In [ ]:
print(X.shape, X_train.shape, X_test.shape)



(151, 6) (121, 6) (30, 6)


ML Model training

Logistic Regression


In [ ]:
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.1)
# define search
search = GridSearchCV(model, grid)
# perform the search
results = search.fit(X_train, Y_train)
# summarizeT
print('MAE:', absolute(results.best_score_))
print('Config: %s' % results.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.666e+04, tolerance: 5.277e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instea

MAE: 0.07531732534187721
Config: {'alpha': 0.0}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.771e+04, tolerance: 5.627e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  

In [ ]:
model1 = Lasso(alpha = 0.0)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model1, X_train, Y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 11.849 (4.774)


In [ ]:
model1.fit(X_train, Y_train)
X_train_prediction = model1.predict(X_train)
a = X_train_prediction.tolist()
X_trainpredictionlist = [math.trunc(a) for a in a]
print(X_trainpredictionlist)
Y_trainlist = Y_train.tolist()
print(Y_trainlist)

[20, 34, 10, 20, 8, 14, 29, 51, 13, 13, 9, 9, 21, 25, 33, 10, 19, 14, 23, 14, 15, 50, 8, 11, 17, 50, 6, 25, 33, 46, 15, 21, 19, 19, 19, 23, 10, 37, 54, 17, 28, 49, 39, 21, 35, 13, 8, 12, 25, 13, 13, 18, 15, 56, 14, 16, 5, 20, 48, 30, 12, 10, 10, 55, 10, 21, 26, 18, 19, 9, 13, 27, 24, 82, 17, 32, 13, 21, 22, 8, 9, 14, 19, 8, 14, 14, 19, 11, 12, 26, 47, 9, 20, 21, 26, 9, 35, 9, 24, 15, 11, 14, 13, 28, 14, 15, 14, 8, 21, 13, 28, 16, 11, 33, 18, 25, 31, 33, 18, 42, 10]
[17, 30, 10, 18, 17, 15, 33, 31, 5, 7, 7, 8, 13, 14, 33, 11, 4, 23, 29, 15, 3, 38, 5, 20, 9, 52, 12, 16, 76, 56, 27, 11, 2, 19, 23, 14, 7, 8, 47, 19, 30, 68, 117, 6, 40, 7, 8, 27, 1, 8, 6, 15, 16, 57, 11, 82, 3, 16, 68, 28, 13, 5, 5, 54, 9, 9, 45, 2, 20, 10, 11, 18, 21, 71, 66, 26, 24, 17, 27, 36, 7, 11, 9, 20, 22, 19, 2, 15, 57, 7, 40, 3, 7, 13, 8, 6, 2, 6, 11, 9, 8, 14, 7, 5, 13, 5, 2, 6, 21, 17, 31, 15, 8, 99, 5, 77, 19, 47, 10, 44, 11]


In [ ]:
# training the Logistic Regression model with Training Data
model.fit(X_train, Y_train)

LinearRegression()

Model Evaluation

In [ ]:
X_train_prediction = model.predict(X_train)
X_train_predictionlist = X_train_prediction.tolist()
Y_trainlist = Y_train.tolist()
print(X_train_predictionlist)
print(Y_trainlist)

[20.779216105536353, 34.84218429669738, 10.965681218512644, 20.51631613393546, 8.004750787863234, 14.467972515959184, 29.019720037169485, 51.129838823018176, 13.768261342741063, 13.482467001935909, 9.983903789705879, 9.684042541417377, 21.40572772494738, 25.103345437982888, 33.47129827005216, 10.333019882174296, 19.413420676591603, 14.946486875467912, 23.668869448163854, 14.091827465358104, 15.137175832029518, 50.74738087308438, 8.737041988624707, 11.645368427815171, 17.84173059625556, 50.23270430898124, 6.1497666955198635, 25.224157090286685, 33.76137783979299, 46.12746376743249, 15.165497719949403, 21.956100300518578, 19.310967675386408, 19.542128436206784, 19.43862779020883, 23.161425103164607, 10.002550936803575, 37.999809033046404, 54.121270694547945, 17.72733360837245, 28.17850123693378, 49.037499695111805, 39.18202306666312, 21.372746708561817, 35.86122708514738, 13.756747920437006, 8.776076771216925, 12.880090118638597, 25.339729694206977, 13.068841444229776, 13.220857645941106

In [ ]:
#accuracy on training data



X_train_predictionlist.index(2)

X_train_predictionlist[0] = 0
X_train_predictionlist[113] = 0
X_train_predictionlist[7] = 0
X_train_predictionlist[8] = 0

Y_trainlist[0] = 0
Y_trainlist[113] = 0
Y_trainlist[7] = 0
Y_trainlist[8] = 0

print(Y_trainlist)
print(X_train_predictionlist)


ValueError: ignored

In [ ]:
training_data_accuracy = mean_squared_error(X_train_predictionlist, Y_trainlist)

print(math.sqrt(training_data_accuracy))

17.107222827873514


In [ ]:
#accuracy on training data

X_test_prediction = model.predict(X_test)
test_data_accuracy = mean_squared_error(X_test_prediction, Y_test)
c = math.sqrt(test_data_accuracy)
print(c)

15.421241005309716


In [ ]:
#accuracy on training data

X_train_prediction = model.predict(X_train)
test_data_accuracy = mean_squared_error(X_test_prediction, Y_test)
b = math.sqrt(test_data_accuracy)
print(b)

15.421241005309716


Predictive System

In [ ]:
seed1 = 1
x = randint(1,100)

In [ ]:
seed = x + 1
value = randint(0,150)
input_data = (X_train[value - 1:value])



test_value = (Y_trainlist[value - 1:value])
#change the input data to a numpy array

input_data_as_numpy_array = np.asarray(input_data)

#reshape the numpy array as we are predicting for only one
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

prediction = model.predict(input_data_reshaped)
if prediction == test_value:

  print('The prediction is correct: Prediction:' , prediction, 'Actual Value:', test_value)


else: strings = [str(integer) for integer in test_value]
a_string = "".join(strings)
a = int(a_string)

x = a - 6
y = a + 6

if prediction in range(x, y, 1):
  print('The prediction is close:', 'Prediction:', prediction, 'Actual Value:', test_value)

else:
  print('The prediction is not close: Prediction:' , prediction, 'Actual Value:', test_value)

The prediction is not close: Prediction: [7] Actual Value: [18]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored